In [1]:
import pickle
import os, numpy
import cv2
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn import neighbors
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

class model:
    def __init__(self):
        print ("initiated")

    def preprocess_data(self,path):
        files = os.listdir(path)
        orb = cv2.ORB_create()
        #sift = cv2.xfeatures2d.SIFT_create()
        X=numpy.zeros(shape=(1,500*32))
        Y=numpy.zeros(shape=(1,))
        count=-1
        for f in files:
             if not (f.startswith('.')):
                folder = os.path.join(path, f)
                filenames = os.listdir(folder)
                count+=1
                for fi in filenames:
                    if not (fi.startswith('.')):
                        filepath = os.path.join(folder, fi)
                        #print filepath
                        image = cv2.imread(filepath, 0)
                        kp, features = orb.detectAndCompute(image,None)
                        #print features.shape
                        features=features.reshape(1,500*32)
                        #print features.shape
                        X = numpy.vstack((X, features))
                        Y=numpy.vstack((Y,numpy.asarray([count])))
        #print X.shape
        Y=Y.reshape(Y.shape[0],)
        return X,Y

    def model_svm(self,X,y):
        X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.33, random_state=42)
        C_score = []
        grid = np.arange(0.01, 3, 0.1)
        for K in grid:
            clf = SVC(C=K)
            # Calculate the mean scores for each value of hyperparameter C
            scores = cross_val_score(clf, X_train, Y_train, cv=5)

            print(scores.mean())
            C_score.append(scores.mean())

        # Display the maximum score achieved at which hyperparameter value
        print (" max score is ", max(C_score), " at C = ", grid[C_score.index(max(C_score))])
        clf = SVC(C=grid[C_score.index(max(C_score))])
        clf.fit(X_train, Y_train)
        y_pred = clf.predict(X_test)
        print("accuracy is {}".format(accuracy_score(Y_test, y_pred)))

    def model_KNN(self,X,Y):
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
        clf = neighbors.KNeighborsClassifier(10)
        clf.fit(X_train, Y_train)
        y_pred = clf.predict(X_test)
        print(accuracy_score(Y_test, y_pred))

    def model_randomforest(self,X,Y):
        print ("printing data shape {}".format(X.shape))

        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)
        K=1000
        clf = RandomForestClassifier(n_estimators=K)
        # Calculate the mean scores for each value of hyperparameter C
        #scores = cross_val_score(clf, X, Y, cv=5)
        #print scores.mean()
        clf.fit(X_train, Y_train)
        y_pred = clf.predict(X_test)
        print(accuracy_score(Y_test, y_pred))




m=model()
X,Y=m.preprocess_data("C:\\Users\\anany\\OneDrive\\Documents\\IIITD\\DPM\\Data")
#m.model_svm(X,Y)
#m.model_KNN(X,Y)
m.model_randomforest(X,Y)

initiated
printing data shape (548, 16000)
0.23636363636363636


In [10]:
# image = cv2.imread('C:\\Users\\anany\\OneDrive\\Documents\\IIITD\\DPM\\Data\\Ancient\\04.jpg')
image = cv2.imread('C:\\Users\\anany\\Downloads\\taj_mahal.jpg')

In [11]:
image.shape

(282, 512, 3)

In [1]:
path ='C:\\Users\\anany\\OneDrive\\Documents\\IIITD\\DPM\\Data\\Ancient'
image_names = os.listdir(path)

In [5]:
path ='C:\\Users\\anany\\OneDrive\\Documents\\IIITD\\DPM\\Git Project\\Templates\\Data\\Ancient'
image_names = os.listdir(path)
string = 'Ancient/'
full_filename = [string+ x for x in image_names]

In [6]:
full_filename[0]

'Ancient/01.jpg'

In [74]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn import neighbors
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import faiss
list = ['Ancient','British','Indo-Islamic','Maratha','Not a Monument', 'Sikh']
with open('C:\\Users\\anany\\OneDrive\\Documents\\IIITD\\DPM\\images.npy', 'rb') as f:
        X= np.load(f)
with open('C:\\Users\\anany\\OneDrive\\Documents\\IIITD\\DPM\\labels.npy', 'rb') as g:     
        Y= np.load(g)

In [75]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

In [76]:
import faiss
def fitModel(train_features): 
   index = faiss.IndexFlatL2(train_features.shape[1])   # build the index 
   index.add(train_features)       # add vectors to the index
   return index
    

In [77]:
X_test= X_test.astype(np.float32)
X_train= X_train.astype(np.float32)
index= fitModel(X_train)

In [78]:
index.ntotal

582

In [79]:
import numpy as np 
#from collections import Counter
from scipy import stats
def predict(train_labels, test_features,k): 
        distance, test_features_faiss_Index = index.search(test_features, k) 
        test_label_faiss_output = stats.mode(train_labels[test_features_faiss_Index],axis=1)[0]
        test_label_faiss_output = np.array(test_label_faiss_output.ravel())
        #for test_index in range(0,test_features.shape[0]):
        #    self.test_label_faiss_output[test_index] = stats.mode(self.train_labels[test_features_faiss_Index[test_index]])[0][0] #Counter(self.train_labels[test_features_faiss_Index[test_index]]).most_common(1)[0][0] 
        return test_label_faiss_output

In [80]:
y_pred = predict(Y_train, X_test, 1)

In [81]:
def getAccuracy(test_label_faiss_output,test_labels):
    accuracy = (test_label_faiss_output == test_labels).mean() 
    return round(accuracy,2) 

In [82]:
getAccuracy(y_pred,Y_test)

0.26

In [71]:
import cv2
import numpy as np
path= "C:\\Users\\anany\\OneDrive\\Documents\\IIITD\\DPM\\Data"
files = os.listdir(path)
orb = cv2.ORB_create()
shape =[]
#sift = cv2.xfeatures2d.SIFT_create()
X=np.zeros(shape=(1,500*32))
Y=np.zeros(shape=(1,))
features= np.array([])
count=-1
for f in files:
    if not (f.startswith('.')):
        folder = os.path.join(path, f)
        filenames = os.listdir(folder)
        count+=1
    for fi in filenames:
        if not (fi.startswith('.')):
            filepath = os.path.join(folder, fi)
            #print filepath
            image = cv2.imread(filepath, 0)
            #print(filepath.split('/')[5])
            kp, features = orb.detectAndCompute(image,None)
            shape = features.shape
            if (shape[0]==500 and shape[1]==32):
                features=features.reshape(1,500*32)
                X = np.vstack((X, features))
                Y=np.vstack((Y,np.asarray([count])))
            else:
                continue
Y=Y.reshape(Y.shape[0],)
    #print X.shape


In [73]:
Y.shape

(647,)

In [70]:
count =0
for i in range(len(shape)):
    if (shape[i][0]==500 and shape[i][1]==32):
       count = count
    else:
        print(i)
        print(shape[i])
        count = count +1

482
(322, 32)


In [62]:
count

0